# Laboratorio 9. EAE253b
<hr/>

**Antes de comenzar:**

- Laboratorio debe ser realizado **de forma individual**. Obviamente, se pueden discutir ideas, pero cualquier intercambio de códigos **no está permitido**.
- Sólo se evaluarán secciones "**TAREAS**"; pero se recomienda realizar secciones **EJERCICIOS**

**Instrucciones de entrega:**

- Debe entregar este laboratorio por WebCurso, en buzón de tareas. Descargar archivo ".ipynb" a su equipo y luego subirlo.
- Además, **debe compartir el Notebook con ayudantes (arybertt@uc.cl, pagonzalez20@uc.cl) y profesor (cealvara@gmail.com)**.
- Plazo máximo de entrega: **Viernes 21 de junio, 11.55 pm.**


<hr />

## Introducción

En este laboratorio, practicaremos con algunas "Tareas" de Machine Learning. En la primera parte, veremos un ejemplo de **clasificación** a través de la regresión logística y luego, veremos cómo realizar lo mismo a través de árboles de decisión.

**Nota: Ejemplos basados en workshop "Machine Learning con Python", realizado por DataUC**

## Parte 0. Importamos algunas herramientas necesarias

In [ ]:
import sqlite3
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sb
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)


Ahora cargamos los datos del lab

In [ ]:
link = 'https://drive.google.com/file/d/11W1Rkz3_4j0YRPX34AZ-nTi6ZFWmTRi-/view?usp=sharing'
id = link.split('/')[5]
print(id)
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('ejemplo.sqlite')

In [ ]:
conn = sqlite3.connect('ejemplo.sqlite')
df = pd.read_sql('SELECT * FROM data where A is not null and b is not null and class is not null', conn)

#veamos algunas estadísticas de esta base de datos
df.describe().transpose()

La base de datos que acabamos de cargar tiene 3 columnas:
- A
- B
- class

In [ ]:
plt.scatter(df['A'], df['B'],alpha=0.5)

## Métodos de clasificación

### Regresión logística

Para estimar un modelo de regresión logística, haremos lo siguiente:

1. Desde el dataframe original, creamos un vector (Serie) que contiene las etiquetas reales de cada dato ("Y") y una matriz de vectores (DataFrame) con las variables explicativas ("X").
2. Luego, necesitamos separar los datos en 2 grupos: un subconjunto de los datos para entrenar el modelo, y un subconjunto para validar nuestros resultados. Usamos train_test_split para separar total de datos en un set de entrenamiento y prueba.
3. El algoritmo de clasificacion que usaremos es una regresion logistica. Importamos desde sklearn la clase LogisticRegression y la entrenamos con los datos de entrenamiento.
4. Ahora podemos visualizar el desempeño de nuestro clasificador comparando las regiones asociadas a cada clase con las etiquetas correctas del set de prueba.
5. Calculamos el score (numero que mide la calidad de nuestro clasificador) u otras métricas (ver acá más metricas: https://scikit-learn.org/stable/modules/classes.html#module-sklearn.metrics)


In [ ]:
from sklearn.preprocessing import StandardScaler
scl = StandardScaler()
X = df.iloc[:, :2].values
Xs = scl.fit_transform(X) #Estandarizamos variables

y = df.iloc[:, 2]

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(Xs, y, test_size = 0.2)

from sklearn.linear_model import LogisticRegression
reg = LogisticRegression().fit(X_train, y_train)

print('Score: {:.3f}'.format(reg.score(X_test, y_test)))

In [ ]:
from sklearn.metrics import confusion_matrix


y_pred = reg.predict(X_test)
conf = confusion_matrix(y_true = y_test, y_pred = y_pred)
_ = plt.matshow(conf, cmap = plt.cm.Blues, alpha = 0.3);
_ = plt.hlines(xmax= 3.5, xmin=-0.5, y=[0.5, 1.5, 2.5]);
_ = plt.vlines(ymax= 3.5, ymin=-0.5, x=[0.5, 1.5, 2.5]);
_ = plt.grid(b = None);
for i in  range(conf.shape[0]):
    for j in range(conf.shape[1]):
        _ = plt.text(x = j, y = i, s = conf[i, j], va = 'center', ha = 'center', fontsize = 20);
        
plt.xlabel('Clase Predicha');
plt.ylabel('Clase Real');

${\displaystyle Precisión\ (precision)={\frac {VP}{VP+FP}}} ;\hspace{0.1cm}  $    

${\displaystyle Sensibilidad\ (recuperacion\ o\ recall)={\frac {VP}{VP+FN}}} ;\hspace{0.1cm} $    

${\displaystyle Exactitud\ (accuracy)={\frac {VP+FN}{Total}}} $    

${\displaystyle Especificidad\ (specificity)={\frac {VN}{FP+VN}}} ;\hspace{0.1cm} $    


<hr/>

### Árbol de decisión


A continuación veremos el caso de los árboles de decisión. Para este ejemplo utilizaremos la "entropía" o información ganada como criterio para determinar la relevancia de cada variable.

En el siguiente link, se puede ver la documentación de este modelo. El parámetro "criterion" permite usar otro criterio para determinar la variable a usar en cada nivel.

https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html#sklearn.tree.DecisionTreeClassifier

In [ ]:
from sklearn import metrics
from sklearn import tree


predictor = DecisionTreeClassifier(criterion="entropy",max_depth=3,min_samples_leaf=10)
predictor = predictor.fit(X_train,y_train)

y_pred = predictor.predict(X_test)

print("Accuracy:",metrics.accuracy_score(y_test, y_pred))
print (predictor.tree_.node_count)

Luego de estimar un "Arbol", podemos visualizar o analizar métricas de interés:

In [ ]:
from sklearn.tree import export_graphviz
from sklearn.externals.six import StringIO  
from IPython.display import Image  
import pydotplus


dot_data = StringIO()
export_graphviz(predictor, out_file=dot_data,  
                filled=True, rounded=True)
graph = pydotplus.graph_from_dot_data(dot_data.getvalue())  
graph.write_png('arbol.png')
Image(graph.create_png())


<hr/>

## Tarea: Aplicación

### Parte 1: Regresión Logística (5 puntos)

Para este ejercicio, cargaremos la base de datos de características de vinos que vimos el lab pasado:

In [ ]:
#Importamos base de datos
bd = datasets.load_wine()
vino = pd.DataFrame(bd.data,columns = ['Alcohol','Malic acid','Ash','Alcalinity of ash','Magnesium','Total phenols','Flavanoids','Nonflavanoid phenols','Proanthocyanins','Color intensity','Hue','OD280/OD315 of diluted wines','Proline'])
categoria = bd.target
vino.head()

### Tareas:

En base a la base de datos cargada, realizando una regresión logística, usted deberá: 

1. Análizar los datos presentados y evaluar la necesidad de reescalar las variables X.
2. De ser necesario, reescalar las variables X.
3. Particionar los datos en 70% para entrenamiento y 30% para posterior prueba.
4. Entrenar su modelo y probarlo. Explique sus resultados.
5. Calcular las 4 métricas antes presentadas.
6. En base a la matriz de confusión: ¿Sería su modelo útil para una compañía que quiere etiquetar una nueva linea como su única premium y hacerla la imagen de la compañía?

In [ ]:
# Ingrese su codigo aqui

### Parte 2: Árboles de decisión (5 puntos)

Para el ejercicio siguiente, usaremos la base de datos de "muestras de tumores" que miramos en clases:

In [ ]:
from sklearn import datasets
cancer = datasets.load_breast_cancer(return_X_y=False)

X = cancer.data
Y = cancer.target

datos = pd.DataFrame(X,columns = list(cancer.feature_names))
categoria = list(cancer.target_names)

datos.head()

Tal como se explicó en clase, esta base de datos contiene diversas características de un tumor puntual, además de una clasificación sobre el tipo de tumor (maligno o benigno). 

Antes de pasar a los ejercicios, corramos un modelo de regresión logistica para analizar los datos y tener un modelo de comparación:

In [ ]:
from sklearn.model_selection import train_test_split
X = StandardScaler().fit_transform(X) #escalamos datos
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.3,random_state = 134) #Separamos

from sklearn.linear_model import LogisticRegression
reg = LogisticRegression().fit(X_train, y_train) #Entrenamos modelo

print('Score: {:.3f}'.format(reg.score(X_test, y_test))) #Imprimimos resultado de la prueba

In [ ]:
from sklearn.metrics import confusion_matrix

y_pred = reg.predict(X_test)
conf = confusion_matrix(y_true = y_test, y_pred = y_pred)   #Se le puede agregar un diseño si la persona quiere.

cm = pd.DataFrame(conf,columns=['P. Maligno','P. Benigno'], index=categoria)
cm

### Tareas:

Utilizando la misma base de datos del ejercicio anterior, usted deberá: 

1. Generar un árbol de decisión con una profundidad de 2 niveles. Grafique.
2. Generar otro árbol de decisión, en el que haya un máximo de 12 observaciones por nodo terminal. Grafique.
3. Generar la matriz de confusión del modelo y en base a los resultados comparar este con la su los resultados de la regresión logística. ¿Qué modelo sería mejor?

In [ ]:
# Ingrese su codigo aqui

<hr/>
<center> <h1>Fin del laboratorio.</h1> </center>

 **Recuerde compartir el Notebook con ayudantes (arybertt@uc.cl, pagonzalez20@uc.cl) y profesor (cealvara@gmail.com)**.